# 🧳 Problema do Caixeiro Viajante (TSP)

O TSP (Travelling Salesman Problem) é um dos problemas mais estudados em ciência da computação e otimização. O objetivo é encontrar o menor ciclo que passe por todas as cidades exatamente uma vez e retorne ao ponto de partida.

- Cada cidade é um vértice.
- Cada caminho entre cidades tem um custo (distância, tempo, etc).
- O desafio é encontrar o ciclo hamiltoniano de menor custo.


## ❓ Por que o TSP é importante?

O TSP aparece em logística, planejamento de rotas, fabricação, robótica, genética, entre outros. Apesar de simples de enunciar, é um problema NP-difícil: não existe algoritmo eficiente conhecido para grandes instâncias.

## 🔎 Métodos para Resolver o TSP

Existem abordagens exatas (garantem o ótimo) e heurísticas (buscam boas soluções rapidamente).


### 💡 Algoritmo Força Bruta (Exato)

Testa todos os caminhos possíveis. Garante a solução ótima, mas é inviável para muitos vértices (complexidade O(n!)).

**Como funciona:**
1. Liste todas as permutações das cidades.
2. Calcule o custo de cada ciclo.
3. Escolha o de menor custo.


In [ ]:
import itertools

def tsp_brute_force(graph, start):
    n = len(graph)
    vertices = list(range(n))
    vertices.remove(start)
    min_path = None
    min_cost = float('inf')
    for perm in itertools.permutations(vertices):
        cost = 0
        k = start
        for j in perm:
            cost += graph[k][j]
            k = j
        cost += graph[k][start]
        if cost < min_cost:
            min_cost = cost
            min_path = (start,) + perm + (start,)
    return min_path, min_cost

# Exemplo
graph = [
    [0, 2, 9, 10, 7],
    [1, 0, 6, 4, 3],
    [15, 7, 0, 8, 3],
    [6, 3, 12, 0, 11],
    [9, 7, 5, 6, 0]
]
path, cost = tsp_brute_force(graph, 0)
print("Melhor ciclo:", path)
print("Custo:", cost)

### 🚴‍♂️ Heurística do Vizinho Mais Próximo (Aproximação Rápida)

Constrói o ciclo escolhendo sempre a cidade mais próxima ainda não visitada. É simples e rápido, mas pode não encontrar o ótimo.

**Como funciona:**
1. Comece em uma cidade.
2. Vá sempre para a cidade mais próxima ainda não visitada.
3. Repita até visitar todas.
4. Retorne ao início.


In [ ]:
def tsp_nearest_neighbor(graph, start):
    n = len(graph)
    visited = [False] * n
    path = [start]
    total_cost = 0
    current = start
    visited[current] = True
    for _ in range(n - 1):
        next_city = None
        min_cost = float('inf')
        for j in range(n):
            if not visited[j] and 0 < graph[current][j] < min_cost:
                min_cost = graph[current][j]
                next_city = j
        path.append(next_city)
        visited[next_city] = True
        total_cost += min_cost
        current = next_city
    total_cost += graph[current][start]
    path.append(start)
    return path, total_cost

# Exemplo
path_nn, cost_nn = tsp_nearest_neighbor(graph, 0)
print("Ciclo pelo vizinho mais próximo:", path_nn)
print("Custo:", cost_nn)

### 🏗️ Heurística da Desigualdade do Triângulo (Aproximação com Garantia)

Se o grafo satisfaz a desigualdade do triângulo (o caminho direto nunca é maior que qualquer caminho indireto), é possível garantir uma solução com custo no máximo o dobro do ótimo.

**Passos:**
1. Encontre uma árvore geradora mínima (MST).
2. Duplique as arestas da MST para criar um grafo euleriano.
3. Encontre um ciclo euleriano.
4. Transforme em ciclo hamiltoniano usando atalhos (pule cidades já visitadas).


In [ ]:
import networkx as nx

def tsp_triangle_heuristic(graph):
    n = len(graph)
    G = nx.Graph()
    for i in range(n):
        for j in range(i+1, n):
            G.add_edge(i, j, weight=graph[i][j])
    mst = nx.minimum_spanning_tree(G)
    eulerian_circuit = list(nx.eulerian_circuit(nx.MultiGraph(mst).to_directed()))
    path = []
    visited = set()
    for u, v in eulerian_circuit:
        if u not in visited:
            path.append(u)
            visited.add(u)
    path.append(path[0])
    cost = sum(graph[path[i]][path[i+1]] for i in range(len(path)-1))
    return path, cost

# Exemplo
path_tri, cost_tri = tsp_triangle_heuristic(graph)
print("Ciclo pela heurística da desigualdade do triângulo:", path_tri)
print("Custo:", cost_tri)

### 🧮 Algoritmo Métrico do Caixeiro Viajante (Aproximação 2-Ótima)

Esse algoritmo segue os passos da heurística anterior, mas é conhecido como algoritmo de Christofides quando implementa também o emparelhamento mínimo de vértices de grau ímpar (não incluído aqui para simplificação).

**Resumo dos passos:**
1. Árvore geradora mínima.
2. Duplica arestas para obter grafo euleriano.
3. Percorre ciclo euleriano.
4. Usa atalhos para obter ciclo hamiltoniano.


In [ ]:
def tsp_metric_algorithm(graph, start=0):
    n = len(graph)
    G = nx.Graph()
    for i in range(n):
        for j in range(i+1, n):
            G.add_edge(i, j, weight=graph[i][j])
    mst = nx.minimum_spanning_tree(G)
    multigraph = nx.MultiGraph()
    multigraph.add_nodes_from(mst.nodes)
    for u, v, data in mst.edges(data=True):
        multigraph.add_edge(u, v, weight=data['weight'])
        multigraph.add_edge(u, v, weight=data['weight'])
    euler_circuit = list(nx.eulerian_circuit(multigraph, source=start))
    hamiltonian_path = []
    visited = set()
    for u, v in euler_circuit:
        if u not in visited:
            hamiltonian_path.append(u)
            visited.add(u)
    hamiltonian_path.append(hamiltonian_path[0])
    total_cost = sum(graph[hamiltonian_path[i]][hamiltonian_path[i+1]] for i in range(len(hamiltonian_path)-1))
    return hamiltonian_path, total_cost

# Exemplo
path_metric, cost_metric = tsp_metric_algorithm(graph, start=0)
print("Ciclo hamiltoniano pelo algoritmo métrico:", path_metric)
print("Custo:", cost_metric)

## 🚚 Aplicações Práticas

- **Logística e transporte:** roteirização de entregas, coleta de lixo, ônibus escolares.
- **Fabricação:** perfuração de placas de circuito, corte de materiais.
- **Robótica:** planejamento de rotas de inspeção.
- **Ciência:** programação de observações astronômicas, sequenciamento genético.

O TSP é um modelo central para muitos problemas reais de otimização!